In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time, datetime 
import re
from openpyxl import load_workbook
# from msoffcrypto_tool import OfficeFile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
import io
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df
# [{"key":"内存","value":"128G"},{"key":"颜色","value":"苍岭绿色"},{"key":"属性","value":"租完归还-可买断"}]

# 近15天iPhone16全系列的机型转化数据 

In [3]:
sql = '''  
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id -- 去除json中的双引号
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag, tojo.app_type, tpmn.name 机型
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id

-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 商品表
left join db_digua_business.t_product tp on tp.id = tod.product_id
-- 商品型号
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id

where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )                 -- 近15天数据
and DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-07-09'
and DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-07-22'
-- and hour(om.create_time) < 18
'''
df_order = query(sql)
df_order.shape


(252339, 49)

In [4]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected

from db_credit.risk
'''
df_risk = query(sql_risk)

In [5]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

In [6]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [7]:
# 挽留数据
sql_wl = '''
select
date_format(om.create_time, '%Y-%m-%d') 下单日期, tort.order_num order_number, tort.task_status
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
'''
df_wl = query(sql_wl)
df_wl.shape

(3612, 3)

In [8]:
df = df_order.copy()
df = df[df.type != 4]
# # 筛选机型为iPhone16的订单
# df = df[df['机型'].str.contains('iPhone16', na=False)]
# 筛选机型为iPhone16的订单
# df = df[df['机型'].str.contains('iPhone', na=False)]
# df.head()


# 处理日期

In [9]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

# 处理备注信息

In [10]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()


df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

颜色
白色钛金属            113116
黑色钛金属             48975
沙漠色钛金属            37331
原色钛金属             16376
白色                13819
                  ...  
蓝色野径回环式表带-S/M         1
海王星                   1
柠檬黄                   1
红白魂                   1
耀黑                    1
Name: count, Length: 157, dtype: int64

内存
256GB          175518
512GB           43081
128GB           22750
12GB+256GB       2283
16GB+512GB       1725
12GB+512GB       1694
256G              852
128G              765
16GB+1TB          595
12GB+1TB          211
128GB WiFi版       139
512G              109
256G WiFi版        108
256GB WiFi版        61
16GB+256GB         32
64G                20
8GB+256GB          15
512GB WiFi版        11
512G WiFi版          8
8GB+128GB           5
64G WiFi版           4
24GB+1TB            3
128G WiFi版          2
256                 2
12GB+256G           2
12G+256GB           1
8G+256GB            1
8GB+128G            1
Name: count, dtype: int64

进件
未进件    202346
进件      49993
Name: count, dtype: int64

# 渠道归属

In [11]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# 订单去重

In [12]:
# 保留不去重的数据
df_no_drop = df.copy()

dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)

# 定义状态

In [13]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# , 'union_rent_tag', 'union_rent_rejected'
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

In [14]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1) 
# df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截']),axis=1) 

# 保留商家的数据

In [15]:
df_contain = df.copy()

# 剔除商家数据

In [16]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)
df.shape

(96599, 77)

# 去重订单明细导出

In [17]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-07-23'

# 导入服务订单

In [18]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]
df_j.shape

(32104, 78)

In [19]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)

df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)


df['是否出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df['是否出库']==1), 1, 0)
df["出库前风控强拒"] = np.where((df["审核状态"]=='出库前风控强拒')&(df['是否出库']==0),1,0)
df["机审强拒"] = np.where((df["审核状态"]=='机审强拒')&(df['是否出库']==0),1,0)
df['机审通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0), 1, 0)
df['风控通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0)&(df['人审拒绝']==0), 1, 0)

df['已退款'] = np.where((df['风控通过件']==1)&(df['审核状态']=='已退款'), 1, 0)

df['是否二手'] = np.where(df['product_name'].str.contains(r'99新|95新|准新|90新'), 1, 0)

df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)

# 筛选二手苹果
# df = df[df['是否二手']==1]

# 导入台账数据

In [20]:
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
# df_tz = pd.read_excel(f_path_ck,sheet_name="2025") 
# df_tz.shape

In [21]:
# dfck = pd.merge(df_tz, df, left_on='订单号', right_on='order_number')
# dfck.drop_duplicates(subset=["订单号"],inplace=True)

In [22]:
# dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# # 删除 露营设备 出库 
# try:
#     dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
# except:
#     dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# # 将序列号转换为日期
# base_date = pd.Timestamp('1899-12-30')  # Excel的基准日期
# dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')

# 每日各个渠道出库单数

In [23]:
# 按转化日期看每日各渠道出库单数
# df_weekday_zh = pd.crosstab(dfck["日期"],dfck["归属渠道"],margins=True)
# # 按下单日期看每日各渠道出库单数
# df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# # 按下单日期看每日各渠道出库单数（纯租物）
# df_ly_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# # 按下单日期看每日支付宝直播商品出库单数
# df_zfb_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# # 按下单日期看 免押 出库单数
# df_my = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)

# 分类sku，spu

In [24]:
df.loc[:,"机型内存"] = df.机型+'_'+df.内存
# 筛选包含iPhone15的订单
# df15 = df[df['机型内存'].fillna('').str.contains('iPhone15')]
# df15[['机型内存','是否二手']]
# df15

# 总体转化

## 剔除拒量订单

In [25]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}


# 二手苹果的转化数据

In [26]:
df_all_iphone = all_models.data_group(df, df2, df_risk_examine, '机型内存')
# df_all_iphone['出库（按下单时间）'] = df_all_iphone['出库']
# df_all_iphone['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
# .sort_values(by='数量',ascending=False)

df_all_iphone = df_all_iphone[["去重订单数","前置拦截","拦截率","进件前取消","进件数","预授权通过率","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消",
                    '出库', '进件出库率','取消率','人审拒绝率','订单出库率']].fillna(0)
# 对每列数据shu进行汇总命名为合计并按降序排列
df_all_iphone.loc['all',:] = df_all_iphone.sum(numeric_only=True)
df_all_iphone_new = df_all_iphone.sort_values(by='去重订单数',ascending=False)
df_all_iphone_new.to_excel(f'F:/需求/瑞彤需求/{t_date}总体各机型内存转化数据.xlsx')


In [28]:
df_all_iphone

,去重订单数,前置拦截,拦截率,进件前取消,进件数,预授权通过率,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,出库,进件出库率,取消率,人审拒绝率,订单出库率
机型内存,,,,,,,,,,,,,,,,
Find X5_8GB+128G,1.0,0.0,0.00%,1.0,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,nan%,0.00%
Find X5_8GB+256GB,1.0,0.0,0.00%,1.0,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,nan%,0.00%
HUAWEI Pura 80 Pro+_16GB+1TB,25.0,12.0,48.00%,7.0,6.0,24.00%,5.0,3.0,2.0,33.33%,0.0,1.0,16.67%,0.00%,50.00%,4.00%
HUAWEI Pura 80 Pro+_16GB+512GB,48.0,17.0,35.42%,22.0,9.0,18.75%,3.0,1.0,2.0,22.22%,0.0,1.0,11.11%,0.00%,11.11%,2.08%
HUAWEI Pura 80 Pro_12GB+1TB,6.0,2.0,33.33%,3.0,1.0,16.67%,1.0,1.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,100.00%,0.00%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
荣耀Magic7 Pro_16GB+512GB,7.0,2.0,28.57%,5.0,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,nan%,0.00%
荣耀Magic7_12GB+256GB,4.0,2.0,50.00%,1.0,1.0,25.00%,0.0,0.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,0.00%,0.00%
荣耀Magic7_12GB+512GB,5.0,2.0,40.00%,3.0,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,nan%,0.00%


In [ ]:

# df_all_sku = all_models.data_group(df, df2, df_risk_examine, '机型内存')
# df_all_sku['出库（按下单时间）'] = df_all_sku['出库']
# # df_all_sku['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
# df_all_sku = df_all_sku[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库（按下单时间）', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_all_sku_new = df_all_sku.reset_index()

df_all_iphon16 = all_models.data_group(df, df2, df_risk_examine, '下单日期')
df_all_iphon16['出库（按下单时间）'] = df_all_iphon16['出库']
df_all_iphon16['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
df_all_iphon16 = df_all_iphon16[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_all_iphon16_new = df_all_iphon16.copy()#.reset_index()

# df_all_iphon16_new.insert(0, '星期', df_all_iphon16_new['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
# df_all_iphon16_new.insert(1, '月份', df_all_iphon16_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_all_iphon16_new['下单日期'].astype(str).str.split('-').str[1])




In [ ]:
for col in df_all_iphon16_new.columns.to_list():
    if df_all_iphon16_new[col].dtype == 'O':
        df_all_iphon16_new.loc[:, col] = pd.to_numeric(df_all_iphon16_new[col].str.replace('%', ''))/100
df_all_iphon16_new.loc['昨日对比',:]=df_all_iphon16_new.diff().iloc[-1]
df_all_iphon16_new.loc['上周对比',:]=df_all_iphon16_new.diff(periods=7).iloc[-2]
df_all_iphon16_new

In [ ]:
df_all_iphon16.index

In [ ]:
# 同时获取近两周的数据，并对比分析
df_iphone_week = week_models.week_data_group_all(df_all_iphon16[["去重订单数","前置拦截", "进件数","机审强拒","机审通过件","人审拒绝","风控通过件","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）']], 7)
df_iphone_week.set_index('week_group', inplace=True)
df_iphone_week

In [ ]:
for col in df_iphone_week.columns.to_list()[1:]:
    
    if df_iphone_week[col].dtype == 'O':
        print(col)
        # df_iphone_week = df_iphone_week.apply(pd.to_numeric, errors='coerce')
        df_iphone_week.loc[:, col] = pd.to_numeric(df_iphone_week[col].str.replace('%', ''))/100


In [ ]:

df_iphone_week.loc['周数据对比',:]=df_iphone_week.diff().iloc[-1]
df_iphone_week

In [ ]:
df_all_spu = all_models.data_group(df, df2, df_risk_examine, '机型')
df_all_spu['出库（按下单时间）'] = df_all_spu['出库']
# df_all_spu['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
df_all_spu = df_all_spu[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库（按下单时间）', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_all_spu_new = df_all_spu.reset_index()

In [ ]:
df_all_spu_new.head()

In [ ]:
# 取iPhone16系列的转化数据，对比昨天的数据和上周的数据

In [ ]:
with pd.ExcelWriter('D:/excel_data/近15天iPhone16转化.xlsx', engine='xlsxwriter') as writer:
    df_all_sku_new.to_excel(writer, sheet_name='按机型内存', index=False)
    df_all_spu_new.to_excel(writer, sheet_name='按机型', index=False)